In [24]:
import requests
from bs4 import BeautifulSoup
import datetime
import re
import pandas as pd
from io import StringIO
import json
import time

In [2]:
adresse_list = ['https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_de_deuxi%C3%A8me_division_2022-2023',
                'https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_de_deuxi%C3%A8me_division_2023-2024',
                'https://fr.wikipedia.org/wiki/Championnat_de_France_de_football_de_deuxi%C3%A8me_division_2024-2025']

In [3]:
season_dict = {}

session = requests.session()
for adresse in adresse_list:
    rankings_dict = {}
    response = session.get(adresse)
    soup = BeautifulSoup(response.text, "html.parser")
    ranking_tables = soup.find_all('table', class_='gauche')
    i = 0
    for table in ranking_tables:
        rankings = table.find_all('tr', class_='notheme')
        rankings_list = []
        for line in rankings:
            ranking_entry = []
            for td in line.find_all('td'):
                if td.find('a'):
                    ranking_entry.append(td.find('a').get('href'))
                    ranking_entry.append(td.find('a').get('title'))
                    ranking_entry.append(td.find('a').get_text().strip('\n'))
                elif td.find('span'):
                    ranking_entry.append(td.find('b').get_text().strip('\n'))
                else:
                    ranking_entry.append(td.get_text().strip('\n'))
            rankings_list.append(ranking_entry)
        if i == 0:
            rankings_dict['TOTAL'] = rankings_list
        elif i == 1:
            rankings_dict['HOME'] = rankings_list
        else:
            rankings_dict['AWAY'] = rankings_list
        i += 1
    season_dict[int(adresse[-4:])-1] = rankings_dict
session.close()

In [4]:
print(season_dict)

{2022: {'TOTAL': [['1', '/wiki/Le_Havre_Athletic_Club_(football)', 'Le Havre Athletic Club (football)', 'Le Havre AC', '75', '38', '20', '15', '3', '46', '19', '+27'], ['2', '/wiki/Football_Club_de_Metz', 'Football Club de Metz', 'FC Metz', '72', '38', '20', '12', '6', '61', '33', '+28'], ['3', '/wiki/Football_Club_des_Girondins_de_Bordeaux', 'Football Club des Girondins de Bordeaux', 'Girondins de Bordeaux', '69', '38', '20', '9', '9', '51', '30', '+21'], ['4', '/wiki/Sporting_Club_bastiais', 'Sporting Club bastiais', 'SC Bastia', '60', '38', '17', '9', '12', '52', '45', '+7'], ['5', '/wiki/Stade_Malherbe_Caen', 'Stade Malherbe Caen', 'SM Caen', '59', '38', '16', '11', '11', '52', '43', '+9'], ['6', '/wiki/En_avant_de_Guingamp', 'En avant de Guingamp', 'EA Guingamp', '55', '38', '15', '10', '13', '51', '46', '+5'], ['7', '/wiki/Paris_Football_Club', 'Paris Football Club', 'Paris FC', '55', '38', '15', '10', '13', '45', '43', '+2'], ['8', '/wiki/Association_sportive_de_Saint-%C3%89tien

In [5]:
club_list_address = []
shortname_list = []
longname_list = []
for season in season_dict:
    ranking_total = season_dict[season]['TOTAL']
    for ranking in ranking_total:
        if ranking[1] not in club_list_address:
            club_list_address.append(ranking[1])
            shortname_list.append(ranking[3])
            longname_list.append(ranking[2])
print(longname_list)

['Le Havre Athletic Club (football)', 'Football Club de Metz', 'Football Club des Girondins de Bordeaux', 'Sporting Club bastiais', 'Stade Malherbe Caen', 'En avant de Guingamp', 'Paris Football Club', 'Association sportive de Saint-Étienne', 'Football Club Sochaux-Montbéliard', 'Grenoble Foot 38', 'Union sportive Quevilly Rouen Métropole', 'Amiens Sporting Club', 'Pau Football Club', 'Rodez Aveyron Football', 'Stade lavallois Mayenne Football Club', 'Valenciennes Football Club', "Football Club d'Annecy", "Dijon Football Côte-d'Or", 'Nîmes Olympique', 'Chamois niortais Football Club', 'Association de la jeunesse auxerroise', "Angers Sporting Club de l'Ouest", 'Sporting Club de Bastia', 'Athletic Club ajaccien', 'Union sportive du littoral de Dunkerque', 'Espérance sportive Troyes Aube Champagne', 'Union sportive concarnoise', 'Football Club de Lorient', 'Clermont Foot 63', 'Red Star Football Club', 'Football Club de Martigues']


In [6]:
len(club_list_address)

31

In [7]:
session = requests.session()
uri = "https://fr.wikipedia.org"
dict_teams = {}
i = 0

for club_address in club_list_address:
    response = session.get(uri + club_address)
    soup = BeautifulSoup(response.text, "html.parser")
    infobox = soup.find('div', class_='infobox_v3')
    fondation = infobox.find('th', string = re.compile('Fondation')).find_next_sibling('td').get_text()
    stade = infobox.find('a', string = re.compile('Stade')).parent.find_next_sibling('td').get_text()
    dict_teams[longname_list[i]] = { 'fondation' : re.search('[0-9]{4}', fondation).group(0), 
                                                        'stade' : re.sub(r' \((.*)\)', '', stade.strip('\n')) }
    i += 1
session.close()
dict_teams

{'Le Havre Athletic Club (football)': {'fondation': '1884',
  'stade': 'Stade Océane'},
 'Football Club de Metz': {'fondation': '1919',
  'stade': 'Stade Saint-Symphorien'},
 'Football Club des Girondins de Bordeaux': {'fondation': '1920',
  'stade': 'Bordeaux Atlantique'},
 'Sporting Club bastiais': {'fondation': '1905',
  'stade': 'Stade Armand-Cesari'},
 'Stade Malherbe Caen': {'fondation': '1913',
  'stade': "Stade Michel-d'Ornano"},
 'En avant de Guingamp': {'fondation': '1912', 'stade': 'Stade de Roudourou'},
 'Paris Football Club': {'fondation': '1969', 'stade': 'Stade Charléty'},
 'Association sportive de Saint-Étienne': {'fondation': '1919',
  'stade': 'Stade Geoffroy-Guichard'},
 'Football Club Sochaux-Montbéliard': {'fondation': '1928',
  'stade': 'Stade Auguste-Bonal'},
 'Grenoble Foot 38': {'fondation': '1911', 'stade': 'Stade des Alpes'},
 'Union sportive Quevilly Rouen Métropole': {'fondation': '1902',
  'stade': 'Stade Robert-Diochon'},
 'Amiens Sporting Club': {'fondat

In [8]:
i = 0
for team in dict_teams:
    dict_teams[team]['shortname'] = shortname_list[i]
    i += 1
dict_teams

{'Le Havre Athletic Club (football)': {'fondation': '1884',
  'stade': 'Stade Océane',
  'shortname': 'Le Havre AC'},
 'Football Club de Metz': {'fondation': '1919',
  'stade': 'Stade Saint-Symphorien',
  'shortname': 'FC Metz'},
 'Football Club des Girondins de Bordeaux': {'fondation': '1920',
  'stade': 'Bordeaux Atlantique',
  'shortname': 'Girondins de Bordeaux'},
 'Sporting Club bastiais': {'fondation': '1905',
  'stade': 'Stade Armand-Cesari',
  'shortname': 'SC Bastia'},
 'Stade Malherbe Caen': {'fondation': '1913',
  'stade': "Stade Michel-d'Ornano",
  'shortname': 'SM Caen'},
 'En avant de Guingamp': {'fondation': '1912',
  'stade': 'Stade de Roudourou',
  'shortname': 'EA Guingamp'},
 'Paris Football Club': {'fondation': '1969',
  'stade': 'Stade Charléty',
  'shortname': 'Paris FC'},
 'Association sportive de Saint-Étienne': {'fondation': '1919',
  'stade': 'Stade Geoffroy-Guichard',
  'shortname': 'AS Saint-Étienne'},
 'Football Club Sochaux-Montbéliard': {'fondation': '192

In [ ]:
df_players_list = []
coach_address_list = []
session = requests.session()

for club_address in club_list_address:
    response = session.get(uri + club_address)
    soup = BeautifulSoup(response.text, "html.parser")
    df = pd.read_html(StringIO(str(soup.select_one("table.sortable[style='margin:0px; width:100%; text-align: center;']"))), extract_links='body')
    df_players_list.append(df)
    try:
        coach_address_list.append(soup.find('a', string='Entraîneur').parent.find_next_sibling('td').find('span').find_next_sibling('a').get('href'))
    except AttributeError:
        coach_address_list.append(soup.find('a', string='Entraîneur').parent.find_next_sibling('td').get_text())

session.close()

In [10]:
coach_address_list

['/wiki/Didier_Digard',
 '/wiki/St%C3%A9phane_Le_Mignan',
 '/wiki/Bruno_Irles',
 '/wiki/Beno%C3%AEt_Tavenot',
 '/wiki/Bruno_Baltazar',
 '/wiki/Sylvain_Ripoll',
 '/wiki/St%C3%A9phane_Gilli',
 '/wiki/Eirik_Horneland',
 '/wiki/Fr%C3%A9d%C3%A9ric_Bompard',
 '/wiki/Franck_Rizzetto',
 '\n David Carré',
 '/wiki/Omar_Daf',
 '/wiki/Nicolas_Usa%C3%AF',
 '/wiki/Didier_Santini',
 '/wiki/Olivier_Frapolli',
 '/wiki/Vincent_Hognon',
 '/wiki/Laurent_Guyot',
 '\n Baptiste Ridira',
 '/wiki/Adil_Hermach',
 '\n Damien Charron',
 '/wiki/Christophe_P%C3%A9lissier_(football)',
 '/wiki/Alexandre_Dujeux',
 '/wiki/Beno%C3%AEt_Tavenot',
 '/wiki/Thierry_Deb%C3%A8s',
 '/wiki/Lu%C3%ADs_Castro_(football,_1980)',
 '/wiki/St%C3%A9phane_Dumont',
 '/wiki/St%C3%A9phane_Rossi',
 '/wiki/Olivier_Pantaloni',
 '/wiki/Laurent_Batlles',
 '/wiki/Gr%C3%A9gory_Poirier',
 '/wiki/Hakim_Malek']

In [11]:
len(df_players_list[0])

1

In [12]:
with open(f"../data/competition/ligue2.json", "w") as file:
    file.write(json.dumps(season_dict, indent=4))

In [13]:
with open(f"../data/teams/ligue2_2022-2024.json", "w") as file:
    file.write(json.dumps(dict_teams, indent=4))

In [14]:
for i in range(len(df_players_list)):
    df_players_list[i][0].to_csv(f"../data/teams/team-{longname_list[i].replace(' ', '_')}.csv")

In [15]:
session = requests.session()
coach_dict = {}
i = 0

for coach in coach_address_list:
    if coach.startswith('/wiki/'):
        uri = "https://fr.wikipedia.org"
        coach_unquote = requests.utils.unquote(uri + coach)
        response = session.get(uri + coach)
        coach_name = re.sub(r' \((.*)\)', '', coach_unquote[30:].replace('_', ' '))
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find(class_='infobox')
        birthdate = table.find('th', string = re.compile('Naissance')).find_next_sibling('td').find('time').get('datetime')
        nationality = table.find('span', string = re.compile('Nationalité')).parent.find_next_sibling('td').get_text()
    else:
        coach_name = coach.replace('\n ', '')
    coach_dict[longname_list[i]] = { 'surname' : coach_name.split()[0], 'name' : ' '.join(coach_name.split()[1:]),
                                     'birthdate' : birthdate, 'nationality' : nationality.strip('\n').replace('   ','/').strip(' ')}
    i += 1

session.close()

In [16]:
with open(f"../data/teams/coaches_ligue2_2022-2024.json", "w") as file:
    file.write(json.dumps(coach_dict, indent=4))

In [51]:
session = requests.session()
match_list = []
i = 0

for i in range(2022,2025):
    season_dict = {}
    season_dict['season'] = f"{i}-{i+1}"
    match_number = 39
    if i == 2024:
        match_number = 35

    for j in range(1,match_number):
        uri = f"https://www.bienpublic.com/sport/calendrier-resultats/saison-{i}-{i+1}/football/national-ligue-2/{j}"
        response = session.get(uri)
        soup = BeautifulSoup(response.text, "html.parser")
        table_list = []
        matchday_dict = {}
        matchday_dict['matchday'] = j

        for table in soup.find_all('table'):
            table_list.append(pd.read_html(StringIO(str(table))))
        for table in table_list:
            date_dict = {}
            date_match = table[0].columns[0]
            date_dict['date'] = date_match
            date_dict['results'] = table[0].set_axis(['home_team', 'score', 'away_team'], axis='columns').to_dict(orient='records')
            try:
                matchday_dict['matches'].extend([date_dict])
            except KeyError:
                matchday_dict['matches'] = [date_dict]
        time.sleep(3)
        try:
            season_dict['season_matches'].extend([matchday_dict])
        except KeyError:
            season_dict['season_matches'] = [matchday_dict]
    # match_dict['season'] = f"{i}-{i+1}"
    match_list.append(season_dict)
    # match_dict['season_matches'] = matchday_dict
            
session.close()

In [52]:
for season in match_list:
    with open(f"../data/matches/matches_ligue2_{season['season']}.json", "w") as file:
        file.write(json.dumps(season, indent=4))